<a href="https://colab.research.google.com/github/jeromepatel/DL-University-Course-Practicals/blob/master/18BCE088_DL_Practical_Stacked_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Apr  5 10:08:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:

#importing libraries
from tensorflow.keras import datasets, utils, models, applications
from keras.datasets.mnist import load_data
from matplotlib import pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, LeakyReLU, Reshape
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Input
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
batch_size = 128
num_classes = 10
epochs = 20


In [9]:
(X_train, Y_train), (X_test, Y_test) = load_data()
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [10]:
X_train = X_train.reshape((X_train.shape[0],784)).astype('float32')
X_test = X_test.reshape((X_test.shape[0],784)).astype('float32')
print(X_train.shape, X_test.shape)


y_train = utils.to_categorical(Y_train)
y_test = utils.to_categorical(Y_test)

X_train = X_train/255
X_test = X_test/255

(60000, 784) (10000, 784)


In [44]:
layer1 = Input(shape=(784,))
layer2 = Dense(100, activation='sigmoid')(layer1)
layer3 = Dense(784, activation='sigmoid')(layer2)

autoencoder1 = models.Model(inputs=layer1, outputs=layer3)

autoencoder1.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_7 (Dense)              (None, 784)               79184     
Total params: 157,684
Trainable params: 157,684
Non-trainable params: 0
_________________________________________________________________


In [45]:
autoencoder1.compile(loss="mean_squared_error",optimizer='adam',metrics=['accuracy'])
autoencoder1.fit(X_train,X_train,batch_size = batch_size,  epochs=10)

Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0839 - accuracy: 0.0111
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0435 - accuracy: 0.0114
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0320 - accuracy: 0.0105
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0250 - accuracy: 0.0115
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0202 - accuracy: 0.0126
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0169 - accuracy: 0.0127
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0146 - accuracy: 0.0124
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0129 - accuracy: 0.0127
Epoch 9/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0116 - accuracy: 0.0120
Epoch 10/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0106 - accuracy: 0.0128

In [47]:
# input_layer = Input(shape=(784,))
input_layer = autoencoder1.layers[0].output
hidden_layer1 = autoencoder1.layers[1].output
trained_embedding_model = keras.Model(inputs=input_layer, outputs=hidden_layer1)
trained_embeddings1 = trained_embedding_model.predict(X_train)
trained_embeddings1.shape

(60000, 100)

In [48]:
layer1 = Input(shape=(100,))
layer2 = Dense(50, activation='sigmoid')(layer1)
layer3 = Dense(100, activation='sigmoid')(layer2)

autoencoder2 = models.Model(inputs=layer1, outputs=layer3)

autoencoder2.summary()

autoencoder2.compile(loss="mean_squared_error",optimizer='adam',metrics=['accuracy'])
autoencoder2.fit(trained_embeddings1,trained_embeddings1,batch_size = batch_size,  epochs=10)

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               5100      
Total params: 10,150
Trainable params: 10,150
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0893 - accuracy: 0.0251
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0546 - accuracy: 0.0423
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0379 - accuracy: 0.0487
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0263 - a

In [49]:
# input_layer = Input(shape=(784,))
input_layer = autoencoder2.layers[0].output
hidden_layer1 = autoencoder2.layers[1].output
trained_embedding_model_2 = keras.Model(inputs=input_layer, outputs=hidden_layer1)
trained_embeddings2 = trained_embedding_model_2.predict(trained_embeddings1)
trained_embeddings2.shape

(60000, 50)

In [50]:
layer1 = Input(shape=(50,))
layer2 = Dense(50, activation='sigmoid')(layer1)
layer3 = Dense(10, activation='sigmoid')(layer2)

autoencoder3 = models.Model(inputs=layer1, outputs=layer3)

autoencoder3.summary()

autoencoder3.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
autoencoder3.fit(trained_embeddings2,y_train,batch_size = batch_size,  epochs=10)

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 50)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                510       
Total params: 3,060
Trainable params: 3,060
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 2.1187 - accuracy: 0.4091
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 1.0876 - accuracy: 0.8231
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.6189 - accuracy: 0.8611
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.4912 - acc

In [51]:
final_model = Sequential()
final_model.add(autoencoder1.layers[0])
final_model.add(autoencoder1.layers[1])
final_model.add(autoencoder2.layers[1])
final_model.add(autoencoder3.layers[1])
final_model.add(autoencoder3.layers[2])
final_model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
final_model.fit(X_train,y_train,epochs=10,batch_size=batch_size)

Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3263 - accuracy: 0.9023
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2184 - accuracy: 0.9348
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1642 - accuracy: 0.9535
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1329 - accuracy: 0.9619
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1142 - accuracy: 0.9680
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0971 - accuracy: 0.9729
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0835 - accuracy: 0.9772
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0701 - accuracy: 0.9810
Epoch 9/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0626 - accuracy: 0.9824
Epoch 10/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0562 - accuracy: 0.9848

In [52]:
final_model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1004 - accuracy: 0.9695


[0.10037020593881607, 0.9695000052452087]